<a href="https://colab.research.google.com/github/lucianocavichioli/ImersaoIA/blob/main/LeitorDeGasometro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
!pip install -q google-generativeai
!pip install -q imap-tools

In [ ]:
"""
At the command line, only need to run once to install the package via pip:
$ pip install google-generativeai
$ pip install imap-tools
"""

from time import sleep
from dataclasses import replace
from google.colab import drive
import google.generativeai as genai
import os
import smtplib
import email.message
from imap_tools import MailBox, AND
from google.colab import userdata
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import json

# minha API KEY
my_key = userdata.get('SECRET_KEY')
genai.configure(api_key=my_key)

# somente o básico do config
my_config = {
  "temperature": 0.5,
  "max_output_tokens": 8192,
}

# inicializa o modelo
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=my_config)

# conecta no googledrive
drive.mount('/content/drive')

# muda o diretorio atual para a pasta do Colab
os.chdir('/content/drive/My Drive/Colab Notebooks/')

# função enviar e-mail avisando que ocorreu um erro
def enviar_email_erro(email, erro):

    corpo_email = "Ocorreu o seguinte erro durante o processamento do seu e-mail:\r\n" + erro

    msg = email.message.Message()
    msg['Subject'] = "Resultado da leitura dos Gasômetros"
    msg['From'] = userdata.get('INPUT_MAIL')
    msg['To'] = email
    password = userdata.get('MAIL_PASSWORD')
    msg.add_header('Content-Type', 'plain')
    msg.set_payload(corpo_email )

    servidor_Gmail = smtplib.SMTP('smtp.gmail.com: 587')
    servidor_Gmail.starttls()
    servidor_Gmail.login(msg['From'], password)
    servidor_Gmail.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    servidor_Gmail.close()
    print('Email de erro enviado')

# função para processar os anexos do e-mail
def processar_anexos(email):

    if len(email.attachments) == 0:
      enviar_email_erro(email.from_, 'O e-mail não possui anexos.')
      return

    historico = []
    for anexo in email.attachments:
      with open(anexo.filename, 'wb') as arquivo_imagem:
          arquivo_imagem.write(anexo.payload)
      historico.append({
               "role": "user",
               "parts": [genai.upload_file(anexo.filename)]
       })

    prepare = model.start_chat(history=historico)
    prepare.send_message("Você é um síndico e precisa identificar os valores atuais dos medidores de gás nas fotos fornecidas.\
       A parte inteira do número no medidor está com o fundo preto e a parte decimal está com o fundo em vermelho.\
       São 4 casas inteiras e 3 casas decimais. A resposta tem que ter exatamente 4 casas inteiras 3 decimais.\
       Também identifique o número do apartamento, que está um uma etiqueta branca, logo abaixo do valor atual do medidor.\
       Formate a saída em um json, informando o 'Apartamento' e 'Leitura")
    enviar_email(prepare.last, email.from_)

# função para buscar novos e-mails no GMail
def buscar_emails():
    my_mail_box = MailBox('imap.gmail.com').login(userdata.get('INPUT_MAIL'), userdata.get('MAIL_PASSWORD'))
    new_emails = my_mail_box.fetch(AND(seen=False))
    for email in new_emails:
      processar_anexos(email)
    my_mail_box.logout

# função para enviar a resposta para o e-mail que enviou as imagens
def enviar_email(resposta, email_destino):

    conteudo_csv = 'Apartamento;Leitura\r\n'
    conteudo_txt = 'AP \tLeitura\r\n'

    conteudo_limpo = resposta.text.replace('```json', '')
    conteudo_limpo = conteudo_limpo.replace('```', '')
    lista = json.loads(conteudo_limpo)
    for leitura in lista:
      conteudo_csv = conteudo_csv + leitura['Apartamento'] + ';' + leitura['Leitura'].replace('.', ',') + '\r\n'
      conteudo_txt = conteudo_txt + leitura['Apartamento'] + '\t' + leitura['Leitura'] + '\r\n'

    # Criar um objeto MIMEMultipart
    msg = MIMEMultipart()
    msg['Subject'] = "Resultado da leitura dos Gasômetros"
    msg['From'] = userdata.get('INPUT_MAIL')
    msg['To'] = email_destino
    password = userdata.get('MAIL_PASSWORD')
    # Adicionar o corpo do e-mail
    msg.attach(MIMEText(conteudo_txt, 'plain'))

    # processa o anexo
    anexo_mime = MIMEBase('application', 'octet-stream')
    # Ler o conteúdo do arquivo e definir como carga do anexo
    anexo_mime.set_payload(conteudo_csv)
    # Codificar o anexo em base64
    encoders.encode_base64(anexo_mime)
    # Adicionar cabeçalhos ao anexo
    anexo_mime.add_header('Content-Disposition', f'attachment; filename="leitura.csv"')
    # Adicionar o anexo ao e-mail
    msg.attach(anexo_mime)

    servidor_Gmail = smtplib.SMTP('smtp.gmail.com: 587')
    servidor_Gmail.starttls()
    servidor_Gmail.login(msg['From'], password)
    servidor_Gmail.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))
    servidor_Gmail.close()
    print('Email de resposta enviado')

while 1:
  buscar_emails()
  # espera 2 minutos e busca novos e-mails
  # verificar esse tempo ou como receber uma notificação quando chegarem novos e-mails
  sleep(120)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
